In [1]:
import pandas as pd
import numpy as np
import pickle

In [14]:
basepath = "/home/jonathan/tesis/11avo_avance/trayectoFeb_Jul_limpio.csv"
data = pd.read_csv(basepath, sep=',',dtype={"fecha": str,
                                        "origen": str,
                                        "destino": str},index_col='fecha',parse_dates=True)          

In [15]:
# data.head(60)
prueba = data.loc[data['periodo']==0]
prueba

,periodo,origen,destino,distancia,viajes
fecha,,,,,
2020-02-14,0,2807905,2807901,002-005,23.751
2020-02-14,0,2807905,2807901,002-005,9.500
2020-02-14,0,2807905,2807901,002-005,60.195
2020-02-14,0,2807905,2807901,002-005,4.718
2020-02-14,0,2807905,2807901,002-005,9.610
...,...,...,...,...,...
2020-07-30,0,2807905,2807901,005-010,17.470
2020-07-31,0,2807905,2807901,005-010,13.193
2020-07-31,0,2807905,2807901,005-010,5.808


In [4]:
# prueba = data.loc[data['fecha']=='2020-03-22']
# prueba = prueba.loc[prueba['periodo']==2]
# prueba

# data=data.loc['2020-02-16':'2020-07-31']
# data.head()

In [5]:
# Obtenemos las fechas de interes 
data.index.freq = data.index.inferred_freq
serieDia = data.resample('24H').sum()
# print(serieDia)
# print(serieDia.index)

In [6]:
data['trayecto'] = data['origen'] + '-' + data['destino']
# data.head(5)

In [7]:
df_SerieTemp = pd.DataFrame({})
reindex=data.reset_index()

for fecha in serieDia.index:   
    filtrar = reindex.loc[reindex.loc[:,'fecha'] == fecha]
#     print(filtrar.shape)
    table_A = pd.pivot_table(filtrar, values=['viajes','origen','destino','fecha'], index=["periodo","trayecto"], aggfunc={'viajes':np.sum,'origen': 'first','destino': 'first','fecha': 'first'})
    reindex2 = table_A.reset_index()
    table_B = pd.pivot_table(reindex2, values=['viajes'], index=["periodo"], columns=["trayecto"], aggfunc={'viajes':'first'}, fill_value=0)
#     print(table_A.shape)
#     print(table_B.shape)
    print(table_B)
    print(table_B.index.shape[0])
    
    #Detectamos errores
    if table_B.index.shape[0] != 24:  # Entra si no tiene las 24 horas
        print(table_B.index.shape[0])
        print(table_B)
        j = 0
        for i in range(24):
            if table_B.index[i] != (i+j):
                print (reindex.fecha.iloc[0],i+j,"origen sin registros!")
                table_B.loc[i+j] = 0
                j += 1
            if (i + j) == 23: break
        table_B = table_B.sort_index() 
    
    table_B = table_B.reindex(range(24), fill_value=0) # Agregamos cero a los datos faltante
    table_B["ds"] = pd.date_range(filtrar.fecha.iloc[0], "%s 23:00:00" % (filtrar.fecha.iloc[0]), freq="H")  #fecha.iloc[0] devuelve "20200214"-"20200215"... y la hora en formato 24h      
    table_B = table_B.set_index("ds") #Fija como indice la columna ds           
    table_B = table_B.astype(np.float32)
    
    df_SerieTemp = df_SerieTemp.append(table_B)
    df_SerieTemp = df_SerieTemp.fillna(0)       
    #break
print(df_SerieTemp.shape)

                  viajes
trayecto 2807905-2807901
periodo                 
0                308.104
1                165.496
2                155.760
3                126.662
4                123.672
5                230.861
6                393.165
7                955.119
8               1226.425
9               1058.345
10              1006.363
11              1129.488
12              1165.153
13              1394.494
14              1626.435
15              1507.759
16              1438.277
17              1257.834
18              1457.480
19              1119.294
20              1114.191
21              1455.890
22              1018.978
23               566.565
24
                  viajes
trayecto 2807905-2807901
periodo                 
0                673.990
1                480.704
2                300.731
3                201.116
4                257.532
5                225.055
6                261.880
7                240.993
8                381.372
9                484.3

/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

                  viajes
trayecto 2807905-2807901
periodo                 
0                715.185
1                453.067
2                237.473
3                181.559
4                179.655
5                140.380
6                209.108
7                199.635
8                271.527
9                381.516
10               609.312
11               860.640
12               854.379
13              1038.029
14               748.368
15               856.359
16               940.837
17               926.283
18               791.167
19               730.042
20               528.704
21               353.474
22               303.167
23               135.798
24
                  viajes
trayecto 2807905-2807901
periodo                 
0                185.730
1                154.664
2                 64.529
3                 89.630
4                170.593
5                175.662
6                424.899
7                953.013
8               1111.551
9               1085.1

/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

                  viajes
trayecto 2807905-2807901
periodo                 
0                153.163
1                102.182
2                 89.874
3                 51.890
4                 70.869
5                201.965
6                384.360
7                938.565
8               1123.745
9               1193.920
10               870.211
11              1010.778
12              1025.292
13              1086.790
14              1492.968
15              1330.768
16              1410.177
17              1525.793
18              2123.564
19              1776.404
20              1252.360
21               720.153
22               422.586
23               209.631
24
                  viajes
trayecto 2807905-2807901
periodo                 
0                195.167
1                 74.244
2                 78.662
3                 51.688
4                101.970
5                132.064
6                417.658
7                958.004
8               1267.419
9               1238.3

...

/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

                  viajes
trayecto 2807905-2807901
periodo                 
0                121.532
1                 90.127
2                 81.083
3                 50.113
4                 31.859
5                111.896
6                249.080
7                310.291
8                511.434
9                516.445
10               434.910
11               602.828
12               568.555
13               635.386
14               697.972
15               607.197
16               727.766
17               546.325
18               726.304
19               615.140
20               580.289
21               368.527
22               268.594
23               159.743
24
                  viajes
trayecto 2807905-2807901
periodo                 
0                212.016
1                136.898
2                 93.963
3                 70.627
4                 25.505
5                115.119
6                122.222
7                404.121
8                431.113
9                488.4

/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

                  viajes
trayecto 2807905-2807901
periodo                 
0                161.104
1                105.788
2                 68.046
3                 54.189
4                 71.142
5                125.282
6                190.218
7                433.890
8                470.853
9                433.528
10               541.664
11               533.390
12               486.902
13               505.456
14               701.320
15               561.559
16               551.741
17               550.686
18               522.007
19               609.115
20               509.437
21               386.472
22               331.242
23               202.452
24
                  viajes
trayecto 2807905-2807901
periodo                 
0                232.085
1                164.182
2                 52.675
3                 46.458
4                 51.251
5                 78.809
6                210.901
7                300.831
8                478.331
9                418.0

/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)
/tmp/ipykernel_5223/449022655.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_SerieTemp = df_SerieTemp.append(table_B)


In [8]:
df_SerieTemp.head()

,viajes
trayecto,2807905-2807901
ds,
2020-02-14 00:00:00,308.104004
2020-02-14 01:00:00,165.496002
2020-02-14 02:00:00,155.759995
2020-02-14 03:00:00,126.662003
2020-02-14 04:00:00,123.671997


In [9]:
df_SerieTemp.columns =[str(s2) for (s1,s2) in df_SerieTemp.columns.tolist()]
df_SerieTemp = df_SerieTemp.reset_index()
print(df_SerieTemp.head())
print(df_SerieTemp.shape)

                   ds  2807905-2807901
0 2020-02-14 00:00:00       308.104004
1 2020-02-14 01:00:00       165.496002
2 2020-02-14 02:00:00       155.759995
3 2020-02-14 03:00:00       126.662003
4 2020-02-14 04:00:00       123.671997
(4056, 2)


In [10]:
prueba = df_SerieTemp.loc[df_SerieTemp['2807905-2807901']==0] # 40 datos quedan como 0 por eliminación de atipicos
prueba.shape

(37, 2)

In [11]:
# df_SerieTemp.to_csv("SerieTempFeb_Jul.csv", index=False)